In [31]:
import panel as pn
from panel.interact import interact
pn.extension('tabulator')

import hvplot.pandas

import holoviews as hv
from holoviews import opts
hv.extension('bokeh')


In [32]:
import pandas as pd
df=pd.read_csv('pokemon_data.csv')
df

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45.0,49.0,49,65.0,65.0,45,1,False
1,2,Ivysaur,Grass,Poison,60.0,62.0,63,80.0,NaN,60,1,False
2,3,Venusaur,Grass,Poison,80.0,NaN,83,100.0,100.0,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,80.0,100.0,123,NaN,120.0,80,1,False
4,4,Charmander,Fire,NaN,NaN,52.0,43,60.0,50.0,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,50.0,100.0,150,100.0,150.0,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,50.0,160.0,110,160.0,110.0,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,80.0,110.0,60,150.0,130.0,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,80.0,160.0,60,170.0,130.0,80,6,True


In [33]:
idf = df.interactive()
HP_slider = pn.widgets.IntSlider(name='HP slider', start=10, end=300, step=3, value=100)
HP_slider

IntSlider(end=300, name='HP slider', start=10, step=3, value=100)

In [34]:
# Radio buttons for CO2 measures
yaxis_pokemon = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['Speed', 'Defense',],
    button_type='success'
)
yaxis_pokemon

RadioButtonGroup(button_type='success', name='Y axis', options=['Speed', 'Defense'], value='Speed')

In [35]:
value = df['Type 1'].unique()

pokemon_pipeline = (
    idf[
        (idf.HP <= HP_slider)
    ]
    .groupby(['Type 1'])[yaxis_co2].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='Type 1')  
    .reset_index(drop=True)
)

In [36]:
pokemon_pipeline

In [37]:
pokemon_plot = co2_pipeline.hvplot(x = 'Type 1', y=yaxis_co2,line_width=2, title="Speed, Defense value depending on HP")
pokemon_plot

In [38]:
pokemon_table = co2_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 10, sizing_mode='stretch_width') 
pokemon_table

In [39]:
df['Type 1'].unique()

array(['Grass', 'Fire', 'Water', 'Bug', 'Normal', 'Poison', 'Electric',
       'Ground', 'Fairy', 'Fighting', 'Psychic', 'Rock', 'Ghost', 'Ice',
       'Dragon', 'Dark', 'Steel', 'Flying'], dtype=object)

In [40]:
df['Type 1'].nunique()

18

In [41]:
filtervalue=['Grass','Fire','Steel']
pokemon_pipeline = (
    idf[
        (idf.HP == HP_slider) &
        (~ (idf['Type 1'].isin(filtervalue)))
    ]
    .groupby(['Type 1', 'Attack', 'Speed'])['Defense'].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='Attack')  
    .reset_index(drop=True)
)
pokemon_pipeline

In [42]:
# Radio buttons for CO2 measures
yaxis_poke = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['Attack','Speed', 'Defense',],
    button_type='success'
)
yaxis_poke

RadioButtonGroup(button_type='success', name='Y axis', options=['Attack', 'Speed', ...], value='Attack')

In [43]:
pokemon_bar_plot = co2_vs_gdp_scatterplot_pipeline.hvplot(kind='bar', 
                                                     x='Type 1', 
                                                     y=yaxis_poke, 
                                                     title='CO2 source by continent')
pokemon_bar_plot

In [44]:
#Create a dashbord

#Layout using Template
template = pn.template.FastListTemplate(
    title='Pokemon DataDashboard', 
    sidebar=[pn.pane.Markdown("# Dashboard creation"), 
             pn.pane.Markdown("#### This dataset contains information on all 802 Pokemon from all Seven Generations of Pokemon. The information contained in this dataset include Base Stats, Performance against Other Types, Height, Weight, Classification, Egg Steps, Experience Points, Abilities, etc. The information was scraped from http://serebii.net/"), 
             pn.pane.PNG('pokemon.png', sizing_mode='scale_both'),
             pn.pane.Markdown("## Settings"),   
             HP_slider],
    main=[pn.Row(pn.Column(yaxis_co2, 
                           pokemon_plot.panel(width=700), margin=(0,25)), 
                 co2_table.panel(width=500)), 
          pn.Row(pn.Column(yaxis_poke, pokemon_bar_plot.panel(width=600)))],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
template.show()
#template.servable();


Launching server at http://localhost:61690
